In [2]:
import tensorflow as tf
tf.__version__

'2.3.0-rc0'

In [3]:
import keras
keras.__version__

'2.4.3'

In [4]:
# !pip install tf2crf

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [16]:
X_train, X_test, y_train, y_test = pd.read_pickle('ner_kor_dataset.pkl')

In [17]:
print('훈련 샘플 문장의 크기 : {}'.format(X_train.shape))
print('훈련 샘플 레이블의 크기 : {}'.format(y_train.shape))
print('테스트 샘플 문장의 크기 : {}'.format(X_test.shape))
print('테스트 샘플 레이블의 크기 : {}'.format(y_test.shape))

훈련 샘플 문장의 크기 : (18492, 125)
훈련 샘플 레이블의 크기 : (18492, 125, 22)
테스트 샘플 문장의 크기 : (4623, 125)
테스트 샘플 레이블의 크기 : (4623, 125, 22)


In [18]:
vocab_size = 46240
tag_size = 22
max_len = 125

---

In [19]:
from tf2crf import CRF
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
# from keras_contrib.layers import CRF

In [20]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=20, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(units=50, return_sequences=True, recurrent_dropout=0.1)))#이전 시점의 단어 정보 뿐만 아니라, 다음 시점의 단어 정보도 참고하기 위함
model.add(TimeDistributed(Dense(50, activation="relu")))  #각 스텝마다 cost (오류)를 계산해서 하위 스텝으로 오류를 전파
crf = CRF(tag_size)
model.add(crf)

# optimizer="adam" 사용

In [7]:
model.compile(optimizer="adam", loss=crf.loss
              , metrics=[crf.accuracy])
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.1, verbose=1)

Epoch 1/10
521/521 [==============================] - 128s 245ms/step - loss: 6.9223 - accuracy: 0.7767 - val_loss: 18.8410 - val_accuracy: 0.8311
Epoch 2/10
521/521 [==============================] - 139s 267ms/step - loss: 3.5390 - accuracy: 0.8449 - val_loss: 17.5451 - val_accuracy: 0.8571
Epoch 3/10
521/521 [==============================] - 159s 305ms/step - loss: 2.5897 - accuracy: 0.8693 - val_loss: 16.4999 - val_accuracy: 0.8674
Epoch 4/10
521/521 [==============================] - 174s 334ms/step - loss: 2.0256 - accuracy: 0.8879 - val_loss: 15.4341 - val_accuracy: 0.8759
Epoch 5/10
521/521 [==============================] - 169s 324ms/step - loss: 1.5995 - accuracy: 0.9032 - val_loss: 14.3467 - val_accuracy: 0.8826
Epoch 6/10
521/521 [==============================] - 169s 325ms/step - loss: 1.2941 - accuracy: 0.9179 - val_loss: 13.3143 - val_accuracy: 0.8932
Epoch 7/10
521/521 [==============================] - 165s 316ms/step - loss: 1.0477 - accuracy: 0.9339 - val_loss: 12

In [8]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

145/145 [==============================] - 13s 92ms/step - loss: 10.9433 - accuracy: 0.9060

 테스트 정확도: 0.9060


In [9]:
model.save("model_biLSTM_adam.h5")

# optimizer="rmsprop" 사용

In [19]:
model.compile(optimizer="rmsprop", loss=crf.loss
              , metrics=[crf.accuracy])
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.1, verbose=1)

Epoch 1/10
521/521 [==============================] - 199s 382ms/step - loss: 6.5845 - accuracy: 0.7916 - val_loss: 16.0977 - val_accuracy: 0.8223
Epoch 2/10
521/521 [==============================] - 201s 385ms/step - loss: 3.8898 - accuracy: 0.8322 - val_loss: 12.2485 - val_accuracy: 0.8492
Epoch 3/10
521/521 [==============================] - 184s 353ms/step - loss: 2.9464 - accuracy: 0.8576 - val_loss: 9.5468 - val_accuracy: 0.8604
Epoch 4/10
521/521 [==============================] - 196s 377ms/step - loss: 2.3490 - accuracy: 0.8712 - val_loss: 8.2265 - val_accuracy: 0.8715
Epoch 5/10
521/521 [==============================] - 202s 388ms/step - loss: 1.9059 - accuracy: 0.8857 - val_loss: 7.7325 - val_accuracy: 0.8808
Epoch 6/10
521/521 [==============================] - 160s 308ms/step - loss: 1.6066 - accuracy: 0.8955 - val_loss: 7.8826 - val_accuracy: 0.8852
Epoch 7/10
521/521 [==============================] - 179s 343ms/step - loss: 1.4248 - accuracy: 0.9018 - val_loss: 8.4429

In [20]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

145/145 [==============================] - 26s 177ms/step - loss: 10.1891 - accuracy: 0.8905

 테스트 정확도: 0.8905


In [21]:
model.save("model_biLSTM.h5")